In [1]:
##Logan Lauton
##Exploratory Stats

##importing required packages for this exploratory stats
import requests
import oracledb
import getpass
import cx_Oracle
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

##connecting to the db as we did in grab_data.ipynb
os.environ.get('TNS_ADMIN')
connection = oracledb.connect(
    user="admin",
    password="aybM6CFL9gykHEKn",
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password="phbUHx6CMjxQj75A")

##print to ensure that connection is successful
print("Successfully connected to Oracle Database")

cursor = connection.cursor()

##selecting everything from the calls table and creating a df
rs1 = cursor.execute("SELECT * FROM CALLS")
df1 = pd.DataFrame(rs1.fetchall())
rs2 = cursor.execute("SELECT * FROM CENSUS")
df2 = pd.DataFrame(rs2.fetchall())
rs3 = cursor.execute("SELECT * FROM HISTORIC_REAL_ESTATE")
df3 = pd.DataFrame(rs3.fetchall())

##print to ensure import of data is successful
print('Successfully imported tables from Oracle Database')

##properly naming the columns
df1.columns = ['Call Number' , 'Date/Time' , 'Address' , 'District' , 'Nature of Call' , 'Status' , 'Latitude' , 'Longitude', "Tract"]
df2.columns = ['GEOID' , 'Tract' , 'Geometry' , 'Estimated Population' , 'Male Population' , 'Female Population' , 'Median Age' , 'Median Income', 'White Population', 'Black Population', 'Native Population', 'Hispanic Population', 'Income Poverty Level']
##cleaning df1
df1 = df1.sort_values(by='Date/Time', ascending=False)
df1[["Tract"]] = df1[["Tract"]].apply(pd.to_numeric)

##cleaing df2
df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]] = df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]].apply(pd.to_numeric)
df2["Calculated Total Population"] = df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]
df2["Percent White"] = round((df2["White Population"] / df2["Calculated Total Population"])*100,3)
df2["Percent Black"] = round((df2["Black Population"] / df2["Calculated Total Population"])*100,3)
df2["Percent Native"] = round((df2["Native Population"] / df2["Calculated Total Population"])*100,3)
df2["Percent Hispanic"] = round((df2["Hispanic Population"] / df2["Calculated Total Population"])*100,3)
df2["Prodominent Tract Population"] = df2[['Percent White', 'Percent Black', 'Percent Native','Percent Hispanic']] \
    .idxmax(axis=1).str.replace('Percent ', '').tolist()

Successfully connected to Oracle Database
Successfully imported tables from Oracle Database


In [34]:
print("Percent White in Milwaukee(Should be 39.9):")
print(round((sum(df2["White Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Black in Milwaukee(Should be 39.4):")
print(round((sum(df2["Black Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Native in Milwaukee(0.6):")
print(round((sum(df2["Native Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Hispanic in Milwaukee(Should be 19.9):")
print(round((sum(df2["Hispanic Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))

print("The percents in the () are given from https://www.census.gov/quickfacts/milwaukeecitywisconsin")

Percent White in Milwaukee(Should be 39.9):
30.734
Percent Black in Milwaukee(Should be 39.4):
14.603
Percent Native in Milwaukee(0.6):
0.288
Percent Hispanic in Milwaukee(Should be 19.9):
54.375
The percents in the () are given from https://www.census.gov/quickfacts/milwaukeecitywisconsin


In [2]:
resultdf=df1.merge(df2,how="left",on="Tract")
resultdf

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,Black Population,Native Population,Hispanic Population,Income Poverty Level,Calculated Total Population,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
0,230910713,04/01/2023 12:59:45 PM,"2323 N MARTIN L KING JR DR,MKE",5,BUSINESS CHECK,Assignment Completed,43.045775,-87.914421,5.507901e+10,014100,...,1167.0,0.0,1489.0,1489.0,2897.0,8.319,40.283,0.000,51.398,Hispanic
1,230910712,04/01/2023 12:59:41 PM,"2357 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,43.061626,-87.947602,5.507901e+10,009900,...,1083.0,0.0,1193.0,1193.0,2297.0,0.914,47.148,0.000,51.937,Hispanic
2,230910060,04/01/2023 12:59:41 AM,"333 W JUNEAU AV,MKE",1,ENTRY,Service in Progress,43.045353,-87.915614,5.507919e+10,186300,...,1630.0,8.0,4424.0,2356.0,8392.0,27.765,19.423,0.095,52.717,Hispanic
3,230901607,04/01/2023 12:59:01 AM,"3352 N 10TH ST,MKE",5,BATTERY,Service in Progress,43.080117,-87.923350,5.507901e+10,006800,...,2118.0,9.0,2259.0,2131.0,4486.0,2.229,47.214,0.201,50.357,Hispanic
4,230910700,04/01/2023 12:58:57 PM,"840 N 24TH ST,MKE",3,WELFARE CITIZEN,Advised,43.041360,-87.942658,5.507901e+10,014800,...,1182.0,30.0,2716.0,2299.0,5143.0,23.624,22.983,0.583,52.810,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40992,230310979,01/31/2023 02:18:31 PM,"1002 W MAPLE ST,MKE",2,TRBL W/SUBJ,Assignment Completed,43.011106,-87.924340,5.507902e+10,016700,...,463.0,53.0,3320.0,3320.0,5497.0,30.216,8.423,0.964,60.397,Hispanic
40993,230310974,01/31/2023 02:17:47 PM,"2300 W HIGHLAND AV,MKE",3,THREAT SCHOOL,Service in Progress,43.045270,-87.941590,5.507901e+10,014800,...,1182.0,30.0,2716.0,2299.0,5143.0,23.624,22.983,0.583,52.810,Hispanic
40994,230310931,01/31/2023 02:16:59 PM,"380 E CAPITOL DR,MKE",5,WELFARE CITIZEN,Unable to Locate Complainant,43.090867,-87.905326,5.507900e+10,004400,...,2003.0,0.0,2872.0,2872.0,5553.0,12.210,36.071,0.000,51.720,Hispanic
40995,230310978,01/31/2023 02:16:01 PM,"3626 W FOND DU LAC AV,MKE",7,SPECIAL ASSIGN,Service in Progress,43.076747,-87.958838,5.507900e+10,004800,...,3124.0,4.0,3496.0,3488.0,6938.0,4.526,45.027,0.058,50.389,Hispanic


In [3]:
empty = resultdf[resultdf['GEOID'].isnull()]
empty

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,Black Population,Native Population,Hispanic Population,Income Poverty Level,Calculated Total Population,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
1457,230901406,03/31/2023 07:03:53 PM,6931 S HOWELL,SPD,TRAFFIC STOP,Advised,42.865506,-87.910120,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1690,230901165,03/31/2023 04:46:18 PM,"500 E OAK ST, OAK CREEK",6,PROPERTY PICKUP,Assignment Completed,42.923150,-87.905010,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1785,230901095,03/31/2023 04:10:25 PM,5970 S HOWELL,SPD,TRAFFIC STOP,City Citation(s) Issued,42.865506,-87.910120,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3654,230880670,03/29/2023 11:56:13 AM,"S 6TH ST / W COLLEGE AV,MKE",6,PROPERTY PICKUP,Assignment Completed,42.930108,-87.919887,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4890,230880837,03/29/2023 01:39:04 PM,"2250 N LYNETTE LN, BROOKFIELD",SPD,ASSIGNMENT,Service in Progress,43.061180,-88.178218,5.513320e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39242,230320659,02/01/2023 11:20:11 AM,D2,2,FOLLOW UP,Assignment Completed,46.109714,-111.166100,3.003100e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39748,230321215,02/01/2023 07:06:23 PM,"8705 N 43RD ST, BROWNDEER WI",5,BATTERY,Service in Progress,43.176117,-87.964664,5.507905e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39858,230321348,02/01/2023 06:02:30 PM,D4,4,PROBATION/PAROLE,Service in Progress,46.109714,-111.166100,3.003100e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40486,230311545,01/31/2023 09:23:22 PM,"8250 N 46TH ST, BROWN DEER",4,VIOL REST ORDER,Assignment Completed,43.168187,-87.968282,5.507905e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
resultdf = resultdf.dropna(subset=['GEOID'])
resultdf

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,Black Population,Native Population,Hispanic Population,Income Poverty Level,Calculated Total Population,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
0,230910713,04/01/2023 12:59:45 PM,"2323 N MARTIN L KING JR DR,MKE",5,BUSINESS CHECK,Assignment Completed,43.045775,-87.914421,5.507901e+10,014100,...,1167.0,0.0,1489.0,1489.0,2897.0,8.319,40.283,0.000,51.398,Hispanic
1,230910712,04/01/2023 12:59:41 PM,"2357 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,43.061626,-87.947602,5.507901e+10,009900,...,1083.0,0.0,1193.0,1193.0,2297.0,0.914,47.148,0.000,51.937,Hispanic
2,230910060,04/01/2023 12:59:41 AM,"333 W JUNEAU AV,MKE",1,ENTRY,Service in Progress,43.045353,-87.915614,5.507919e+10,186300,...,1630.0,8.0,4424.0,2356.0,8392.0,27.765,19.423,0.095,52.717,Hispanic
3,230901607,04/01/2023 12:59:01 AM,"3352 N 10TH ST,MKE",5,BATTERY,Service in Progress,43.080117,-87.923350,5.507901e+10,006800,...,2118.0,9.0,2259.0,2131.0,4486.0,2.229,47.214,0.201,50.357,Hispanic
4,230910700,04/01/2023 12:58:57 PM,"840 N 24TH ST,MKE",3,WELFARE CITIZEN,Advised,43.041360,-87.942658,5.507901e+10,014800,...,1182.0,30.0,2716.0,2299.0,5143.0,23.624,22.983,0.583,52.810,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40992,230310979,01/31/2023 02:18:31 PM,"1002 W MAPLE ST,MKE",2,TRBL W/SUBJ,Assignment Completed,43.011106,-87.924340,5.507902e+10,016700,...,463.0,53.0,3320.0,3320.0,5497.0,30.216,8.423,0.964,60.397,Hispanic
40993,230310974,01/31/2023 02:17:47 PM,"2300 W HIGHLAND AV,MKE",3,THREAT SCHOOL,Service in Progress,43.045270,-87.941590,5.507901e+10,014800,...,1182.0,30.0,2716.0,2299.0,5143.0,23.624,22.983,0.583,52.810,Hispanic
40994,230310931,01/31/2023 02:16:59 PM,"380 E CAPITOL DR,MKE",5,WELFARE CITIZEN,Unable to Locate Complainant,43.090867,-87.905326,5.507900e+10,004400,...,2003.0,0.0,2872.0,2872.0,5553.0,12.210,36.071,0.000,51.720,Hispanic
40995,230310978,01/31/2023 02:16:01 PM,"3626 W FOND DU LAC AV,MKE",7,SPECIAL ASSIGN,Service in Progress,43.076747,-87.958838,5.507900e+10,004800,...,3124.0,4.0,3496.0,3488.0,6938.0,4.526,45.027,0.058,50.389,Hispanic


In [5]:
#patrol call per tract for majority white neighborhoods vs other - check for different h0 m1==m2, with very low p value.
patroldata = resultdf[resultdf['Nature of Call'] == "PATROL"]
patroldata

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,Black Population,Native Population,Hispanic Population,Income Poverty Level,Calculated Total Population,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
23,230910068,04/01/2023 12:50:52 AM,"N HOPKINS ST / W GLENDALE AV,MKE",7,PATROL,Assignment Completed,43.100776,-87.958364,5.507900e+10,002600,...,2325.0,27.0,2671.0,2663.0,5270.0,4.687,44.118,0.512,50.683,Hispanic
31,230910696,04/01/2023 12:46:25 PM,"1800 N 2ND ST,MKE",5,PATROL,Assignment Completed,43.053962,-87.912653,5.507901e+10,011400,...,478.0,0.0,1666.0,1666.0,3291.0,34.853,14.524,0.000,50.623,Hispanic
42,230910054,04/01/2023 12:40:17 AM,"N 60TH ST / W BIRCH AV,MKE",4,PATROL,Assignment Completed,43.117238,-87.986212,5.507900e+10,001900,...,2797.0,0.0,3310.0,3310.0,6394.0,4.489,43.744,0.000,51.767,Hispanic
71,230910032,04/01/2023 12:25:46 AM,"2745 S HOWELL AV,MKE",6,PATROL,Assignment Completed,42.994612,-87.904592,5.507902e+10,018400,...,0.0,0.0,1439.0,1425.0,2818.0,48.935,0.000,0.000,51.065,Hispanic
72,230910031,04/01/2023 12:25:24 AM,"N 53RD ST / W CENTER ST,MKE",3,PATROL,Assignment Completed,43.067993,-87.979848,5.507901e+10,005900,...,2590.0,0.0,3764.0,3684.0,7243.0,12.274,35.759,0.000,51.967,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40678,230311441,01/31/2023 06:26:38 PM,"N PALMER ST / E CHAMBERS ST,MKE",5,PATROL,Assignment Completed,43.052670,-87.909770,5.507901e+10,011400,...,478.0,0.0,1666.0,1666.0,3291.0,34.853,14.524,0.000,50.623,Hispanic
40688,230311425,01/31/2023 06:16:06 PM,"500 S 1ST ST,MKE",2,PATROL,Assignment Completed,43.008533,-87.911185,5.507902e+10,016600,...,377.0,19.0,1996.0,1996.0,2978.0,19.678,12.660,0.638,67.025,Hispanic
40751,230311327,01/31/2023 05:25:43 PM,"2302 W CAPITOL DR,MKE",5,PATROL,Assignment Completed,43.089978,-87.941111,5.507900e+10,004200,...,3742.0,0.0,4144.0,4139.0,8093.0,2.558,46.237,0.000,51.205,Hispanic
40770,230311300,01/31/2023 05:11:25 PM,3200 N 27TH ST,5,PATROL,Assignment Completed,43.077573,-87.947132,5.507901e+10,006500,...,2289.0,17.0,2475.0,2475.0,4836.0,1.137,47.333,0.352,51.179,Hispanic


In [6]:
tracts_w_patrols = df2[df2['Tract'].isin(patroldata.groupby(['Tract']).size().sort_values(ascending = False).index.tolist())]
tracts_w_patrols

,GEOID,Tract,Geometry,Estimated Population,Male Population,Female Population,Median Age,Median Income,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Calculated Total Population,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
1,008800,55079008800,POLYGON ((-65976.74531288422 4791487.360095243...,1620.0,795.0,825.0,32.5,16713.0,90.0,1498.0,4.0,1620.0,1620.0,3212.0,2.802,46.638,0.125,50.436,Hispanic
2,008500,55079008500,POLYGON ((-64585.292365803034 4791763.78518607...,1540.0,756.0,784.0,29.5,19546.0,6.0,1517.0,17.0,1540.0,1540.0,3080.0,0.195,49.253,0.552,50.000,Hispanic
4,008900,55079008900,POLYGON ((-66590.0980055522 4791541.1265165135...,1548.0,724.0,824.0,21.4,12663.0,95.0,1268.0,0.0,1548.0,1444.0,2911.0,3.263,43.559,0.000,53.178,Hispanic
5,009000,55079009000,POLYGON ((-67253.65211980906 4790705.913242541...,2008.0,740.0,1268.0,22.3,16311.0,64.0,1623.0,0.0,2008.0,1966.0,3695.0,1.732,43.924,0.000,54.344,Hispanic
6,009100,55079009100,POLYGON ((-67849.48754489853 4791240.738696917...,1930.0,905.0,1025.0,31.3,13194.0,154.0,1531.0,0.0,1930.0,1930.0,3615.0,4.260,42.351,0.000,53.389,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,002600,55079002600,POLYGON ((-67066.56444233644 4795193.581384823...,2671.0,1103.0,1568.0,29.1,18661.0,247.0,2325.0,27.0,2671.0,2663.0,5270.0,4.687,44.118,0.512,50.683,Hispanic
281,002800,55079002800,POLYGON ((-67761.96733339364 4796157.683783457...,2459.0,1169.0,1290.0,26.3,18570.0,147.0,2113.0,5.0,2459.0,2387.0,4724.0,3.112,44.729,0.106,52.053,Hispanic
282,003000,55079003000,POLYGON ((-70195.59329969867 4796399.667200316...,4507.0,2163.0,2344.0,26.7,27378.0,417.0,3465.0,2.0,4507.0,4507.0,8391.0,4.970,41.294,0.024,53.712,Hispanic
283,003100,55079003100,POLYGON ((-69395.83786693972 4795469.244458136...,3862.0,1719.0,2143.0,31.6,21779.0,478.0,2835.0,25.0,3862.0,3845.0,7200.0,6.639,39.375,0.347,53.639,Hispanic


In [17]:
df2.groupby(['Prodominent Tract Population']).size().sort_values(ascending = False)

Prodominent Tract Population
Hispanic    291
Black         1
dtype: int64